## Preparing data

In [1]:
import urllib.request

url = ("https://raw.githubusercontent.com/rasbt/"
"LLMs-from-scratch/main/ch02/01_main-chapter-code/"
"the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x107eab250>)

In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total characters in text:", len(raw_text))
print(raw_text[:100])

Total characters in text: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [3]:
import re
text = "Hello, world. This, is a test."
result = re.split(r'([,.]|\s)', text)
print(result)

# or
result = [item for item in result if item.strip()]
print(result)


['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']
['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [4]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [5]:
# apply to full text

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

print(preprocessed[:30])

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [6]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [7]:
vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [8]:
# implementing a simple text tokenizer

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
        item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [9]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [10]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [11]:
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

# will throw key error as hello is not in vocab

KeyError: 'Hello'

In [12]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}
print(len(vocab.items()))

1132


In [13]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [14]:
# implementing a simple text tokenizer

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
        item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [item if item in self.str_to_int
        else "<|unk|>" for item in preprocessed] # replace unknown words
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text) # remove extra spaces
        return text
    
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [15]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [16]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


In [17]:
# Byte Pair Encoding

from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.8.0


In [18]:
# use the tokenizer pre-built for GPT-2
tokenizer = tiktoken.get_encoding("gpt2")
text = (
"Hello, do you like tea? <|endoftext|> In the sunlit terraces"
"of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [19]:
strings = tokenizer.decode(integers)
print(strings)

# it can process unknow words and each token is a subword unit

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [20]:
text = "Akwirw ier"

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

strings = tokenizer.decode(integers)
print(strings)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


In [21]:
# data sampling with a sliding window

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [22]:
enc_sample = enc_text[50:] # remove for demonstration purposes

context_size = 4
x = enc_sample[:context_size] # input tokens
y = enc_sample[1:context_size+1] # target token
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [23]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [24]:

for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [25]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt) # tokenizes the entire text
        
        for i in range(0, len(token_ids) - max_length, stride):
            # use sliding window to chunk the book into overlapping sequences of max_length
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids) # returns total rows in the dataset
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx] # return single row
        
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True, num_workers=0):
    
    tokenizer = tiktoken.get_encoding("gpt2") # initialize the tokenizer

    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride) # create the dataset

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, # create the dataloader
                            drop_last=drop_last, num_workers=num_workers)
    
    return dataloader

/Users/dani/Library/Caches/pypoetry/virtualenvs/lit-llm-usCjbgK0-py3.11/lib/python3.11/site-packages/torch/_subclasses/functional_tensor.py:295: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [26]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader) # converts the dataloader to an iterator to fetch the next entry via next()
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [27]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [28]:
# bigger batch size
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4,shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [29]:
# creating token embeddings

vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length,stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [30]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [31]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([4, 256])
torch.Size([8, 4, 256])


In [32]:
print(token_embeddings)

tensor([[[ 0.6674,  0.3533, -0.6187,  ..., -0.3895, -0.2808,  0.8029],
         [-0.2430,  1.3017,  0.5219,  ..., -0.0933, -0.2718,  1.3389],
         [-0.3922, -0.8160, -0.8324,  ...,  0.6634,  0.8036,  0.7926],
         [-0.2741, -2.6191,  1.3744,  ...,  0.9238, -1.2528,  0.8276]],

        [[-0.1567, -0.9246, -0.0746,  ...,  1.2055, -0.3228,  1.0254],
         [ 0.2522, -0.2396, -1.1664,  ...,  0.6438,  0.5554, -0.4259],
         [ 0.3906,  0.6300,  0.1811,  ...,  1.4865, -0.6195, -1.0222],
         [-0.2656,  1.2207, -0.2841,  ...,  0.6723, -0.4774,  1.2391]],

        [[ 1.3327, -0.5670,  0.8645,  ..., -0.7654,  0.0130,  1.2400],
         [ 0.0528,  0.3682, -1.6151,  ...,  0.2551,  0.2737, -0.2941],
         [ 0.5267, -0.9816,  0.4868,  ...,  0.6460,  0.7574,  0.3339],
         [ 0.5467, -1.6024, -0.6598,  ..., -0.2744,  0.0117, -0.3925]],

        ...,

        [[ 0.4723, -0.4663, -0.6287,  ...,  0.1788,  0.1929,  1.8765],
         [ 1.0319, -0.2114,  2.1420,  ..., -1.0037, -0.12

## Attention mechanism

In [33]:
import torch
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55]] # step (x^6)
)

query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [34]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


In [35]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0) # using softmax

attn_weights_2_naive = softmax_naive(attn_scores_2)
print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [36]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0) # better softmax implementation
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [37]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i
print("Context vector:", context_vec_2)

Context vector: tensor([0.4419, 0.6515, 0.5683])


In [38]:
### computing attention weights for all input tokens

attn_scores = torch.empty(6,6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i,j] = torch.dot(x_i, x_j)
print(attn_scores)

attn_scores = inputs @ inputs.T # same as above but more efficient
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [39]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [40]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)
print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [41]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [42]:
# implement trainable weights: self-attention
# first, step by step

# let's compute only one context vector for now:

x_2 = inputs[1] # query as the second token
d_in = inputs.shape[1] # input dimension
d_out = 2 # output dimension

# initialize the weight matrices

torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) # query, grad as False to not update
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) # key
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) # value

query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print("Query:", query_2)

Query: tensor([0.4306, 1.4551])


In [43]:
keys = inputs @ W_key
values = inputs @ W_value
print("Keys shape:", keys.shape)
print("Values shape:", values.shape)

# if you see, we just projected the input tokens to a lower dimension (3D -> 2D)

Keys shape: torch.Size([6, 2])
Values shape: torch.Size([6, 2])


In [44]:
keys_2 = keys[1]
attn_scores_22 = torch.dot(query_2, keys_2) # or query_2.dot(keys_2)
print("Attention scores:", attn_scores_22)

attn_scores_2 = query_2 @ keys.T # generalize to all attention scores
print("Attention scores:", attn_scores_2)

Attention scores: tensor(1.8524)
Attention scores: tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [45]:
d_k = keys.shape[-1] # scaling factor
attn_weights_2 = torch.softmax(attn_scores_2 / (d_k ** 0.5), dim=-1)
# scaling by sqrt(d_k) to stabilize the gradients and avoid vanishing gradients
print("Attention weights:", attn_weights_2)

Attention weights: tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [46]:
context_vec_22 = attn_weights_2 @ values
print("Context vector:", context_vec_22)

Context vector: tensor([0.3061, 0.8210])


In [47]:
# secondly, let's now implement it as a compact class

import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__() # initialize the base class
        # Initialize trainable weight matrices for query, key, and value projections
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        # Step 1: Project input into query, key, and value spaces
        keys = x @ self.W_key  # Shape: (n_tokens, d_out)
        queries = x @ self.W_query  # Shape: (n_tokens, d_out)
        values = x @ self.W_value  # Shape: (n_tokens, d_out)
        
        # Step 2: Compute attention scores
        attn_scores = queries @ keys.T  # Shape: (n_tokens, n_tokens)
        
        # Step 3: Apply scaling and softmax to get attention weights
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        
        # Step 4: Use attention weights to compute context vectors
        context_vec = attn_weights @ values  # Shape: (n_tokens, d_out)
        
        return context_vec

In [48]:
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [49]:
d_in, d_out = 3, 2
n_tokens = 6

# Input: Random tensor of shape (n_tokens, d_in)

# Initialize and test SelfAttention
torch.manual_seed(123)
self_attention = SelfAttention(d_in, d_out)
context_vectors = self_attention(inputs)

print(context_vectors)  # Should be (n_tokens, d_out)


tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [50]:
torch.manual_seed(123)
sa_v1 = SelfAttention(d_in=3, d_out=2)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [51]:
# leverage nn.Linear layers to replace the manual weight initialization

class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
        attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        context_vec = attn_weights @ values
        return context_vec
    
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [52]:
# Example reproducing same results but using V2 weights in V1
torch.manual_seed(789)

# Define dimensions and create a sample input
d_in    = 3       # Input feature dimension
d_out   = 2       # Output feature dimension
n_tokens = 6     # Number of tokens (or sequence length)
inputs = inputs  # Sample input

# Create an instance of SelfAttention_v2 and SelfAttention_v1
attn_v2 = SelfAttention_v2(d_in, d_out, qkv_bias=False)
attn_v1 = SelfAttention(d_in, d_out)

# Transfer weights from v2 to v1.
# Note: nn.Linear in v2 has weights of shape (d_out, d_in)
# while in v1 the weights are stored as (d_in, d_out), so we transpose.
with torch.no_grad():
    attn_v1.W_query.copy_(attn_v2.W_query.weight.data.T)
    attn_v1.W_key.copy_(attn_v2.W_key.weight.data.T)
    attn_v1.W_value.copy_(attn_v2.W_value.weight.data.T)

# Now compute the context vectors using SelfAttention_v1 with v2 weights.
context_vectors = attn_v1(inputs)

print("Context Vectors:")
print(context_vectors)
print("Shape:", context_vectors.shape)  

Context Vectors:
tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)
Shape: torch.Size([6, 2])


In [53]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55]] # step (x^6)
)



In [54]:
# hiding future words with causal attention

# reuse previous attention weights
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length), diagonal=0)
print(mask_simple)

masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)
tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [55]:
# renormalize again

row_sums = masked_simple.sum(dim=-1, keepdim=True) # keep dim True does 
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

# After masking, we renormalize the attention weights
# so that only unmasked positions contribute to the final softmax distribution. 
# This process effectively recalculates the softmax over the unmasked tokens, 
# ensuring that no information from the masked (future) tokens influences the result.

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


In [56]:
# now more efficient by using the math property of softmax
# when -inf are presente, softmax will treat them as 0's
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [57]:
# masking additional attention weights with dropout

torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # 50% dropout rate e.g.
example = torch.ones(6, 6)
print(dropout(example))

# note that elements are scaled by 1/(1-p) ensuring avg influence remains consistent

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [58]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


In [59]:
# lets now implement a compact causal attention class

batch = torch.stack([inputs, inputs], dim=0)
print(batch.shape) # 2 inputs with 6 tokens each, each token has embedding dim = 3

torch.Size([2, 6, 3])


In [60]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, qkv_bias)
        self.dropout = nn.Dropout(dropout) ## add dropout layer
        self.register_buffer( # register buffer is for moving to cpu/gpu
            "mask",
             torch.triu(torch.ones(context_length, context_length),
            diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape # batch size, num tokens, input dim
        keys = self.W_key(x) # key is projected
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # transpose to match dims, keep batch dim
        attn_scores.masked_fill( # in pytorch, inplace ops are done for memory efficiency
            self.mask[:num_tokens, :num_tokens].bool(), -torch.inf
        )
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values
        return context_vec
    
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, dropout=0.0)
context_vectors = ca(batch)
print("context_vectors:", context_vectors)
print("context_vectors shape:", context_vectors.shape)

context_vectors: tensor([[[-0.5337, -0.1051],
         [-0.5323, -0.1080],
         [-0.5323, -0.1079],
         [-0.5297, -0.1076],
         [-0.5311, -0.1066],
         [-0.5299, -0.1081]],

        [[-0.5337, -0.1051],
         [-0.5323, -0.1080],
         [-0.5323, -0.1079],
         [-0.5297, -0.1076],
         [-0.5311, -0.1066],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vectors shape: torch.Size([2, 6, 2])


In [61]:
# extending to multi-head attention
# let's start by stacking multiple attention heads

class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length,
                 dropout, n_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList([
            CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
            for _ in range(n_heads)
        ])
    
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

torch.manual_seed(123)
context_length = batch.shape[1] # 6 tokens
d_in, d_out = 3, 2

mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, dropout=0.0, n_heads=2)
context_vectors = mha(batch)

print("context_vectors:", context_vectors)
print("context_vectors shape:", context_vectors.shape)

context_vectors: tensor([[[-0.5337, -0.1051,  0.5085,  0.3508],
         [-0.5323, -0.1080,  0.5084,  0.3508],
         [-0.5323, -0.1079,  0.5084,  0.3506],
         [-0.5297, -0.1076,  0.5074,  0.3471],
         [-0.5311, -0.1066,  0.5076,  0.3446],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.5337, -0.1051,  0.5085,  0.3508],
         [-0.5323, -0.1080,  0.5084,  0.3508],
         [-0.5323, -0.1079,  0.5084,  0.3506],
         [-0.5297, -0.1076,  0.5074,  0.3471],
         [-0.5311, -0.1066,  0.5076,  0.3446],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vectors shape: torch.Size([2, 6, 4])


In [62]:
# sample for d_out = 1

d_out = 1

mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, dropout=0.0, n_heads=2)
context_vectors = mha(batch)

print("context_vectors:", context_vectors)
print("context_vectors shape:", context_vectors.shape)

context_vectors: tensor([[[0.2761, 0.2520],
         [0.2765, 0.2514],
         [0.2765, 0.2514],
         [0.2745, 0.2513],
         [0.2752, 0.2514],
         [0.2748, 0.2513]],

        [[0.2761, 0.2520],
         [0.2765, 0.2514],
         [0.2765, 0.2514],
         [0.2745, 0.2513],
         [0.2752, 0.2514],
         [0.2748, 0.2513]]], grad_fn=<CatBackward0>)
context_vectors shape: torch.Size([2, 6, 2])


In [64]:
# now, with weight-splits combining both the wrapper and causal attention
# multi-head attention class should split the input into multiple heads

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out,
                context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"
        
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # reduces the projection dim to match desired output dim
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out) # uses linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) # split matrix adding num_heads dim
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim) # Then we unroll the last dim: d_out -> num_heads, head_dim
        values = values.view(b, num_tokens, self.num_heads, self.head_dim) # output (b, num_tokens, num_heads, head_dim)

        keys = keys.transpose(1, 2) # transpose from b, num_tokens, num_heads, head_dim -->
        queries = queries.transpose(1, 2) # b, num_heads, num_tokens, head_dim
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2) # transpose back to b, num_tokens, num_heads, head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out) # reshape to b, num_tokens, d_out
        context_vec = self.out_proj(context_vec) # adds optional linear projection
        return context_vec

In [68]:
batch.shape

torch.Size([2, 6, 3])

In [70]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2

mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print(context_vecs.shape)


tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
torch.Size([2, 6, 2])


In [71]:
# For illustration, create a dummy batch using GPT-2-like dimensions:
batch_size = 2            # You can choose any batch size; here we use 2.
context_length = 10       # Let's say each sequence has 10 tokens.
d_in = 768                # GPT-2 input embedding size.
d_out = 768               # GPT-2 output embedding size.
num_heads = 12            # GPT-2 smallest model uses 12 attention heads.

# Create a dummy input batch of shape (batch_size, context_length, d_in)
batch = torch.randn(batch_size, context_length, d_in)

# Initialize the MultiHeadAttention module with the specified parameters.
mha = MultiHeadAttention(d_in, d_out, context_length, dropout=0.0, num_heads=num_heads)

# Pass the batch through the multi-head attention module.
context_vecs = mha(batch)

# Print the resulting context vectors and their shape.
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[ 5.6764e-01, -1.4319e-02,  5.1594e-01,  ...,  1.7414e-01,
           2.3350e-01,  4.8068e-04],
         [ 3.8421e-01, -9.5690e-02,  3.6479e-01,  ..., -1.2758e-01,
           1.2907e-01,  1.4481e-01],
         [ 1.3668e-01,  7.4592e-02,  2.4618e-01,  ..., -9.9234e-02,
           2.7782e-01,  2.2877e-02],
         ...,
         [ 3.7984e-02, -2.3183e-02,  5.3379e-02,  ..., -4.5788e-02,
           7.6621e-02,  1.1719e-02],
         [ 2.7453e-02,  6.5239e-02,  3.8844e-02,  ..., -1.0310e-02,
           1.3317e-02, -5.7621e-02],
         [ 5.2384e-02,  2.9487e-02,  1.0222e-01,  ..., -5.0533e-03,
          -1.5477e-02, -1.0044e-01]],

        [[-1.9404e-01, -2.6481e-01,  4.6185e-02,  ..., -2.3721e-01,
           1.3498e-01,  4.8691e-02],
         [ 1.4757e-01, -1.2062e-01, -1.5164e-01,  ..., -1.8326e-01,
          -5.3520e-02, -9.1627e-02],
         [ 8.9989e-02, -8.5140e-02, -4.3592e-03,  ..., -1.7663e-01,
          -2.0616e-01,  3.7935e-01],
         ...,
         [ 1.3818e-01,  9